In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
import pickle
import time
%matplotlib inline

In [2]:
p_path="/home/takeyama/pywork/ipython/2016-05-30/"

In [3]:
cd ~/Documents/SyncRecord/cleaning-addingLABEL/

/home/takeyama/Documents/SyncRecord/cleaning-addingLABEL


クラスの問題について、レビューを頂いたのでメモする。
・オブジェクトをセンサデータ１つにしたことに対して問題がある。
センサ間のJS、KLダイバージェンスを調べるときに内部のインスタンスを呼ばないと大変
→１つのクラスですべてのセンサデータを扱える方がいい

問題に対して、クラスを位置から設計するのは大変
・クラスをまとめるクラスを用意する必要がある。

In [4]:
class AllSensorData:        
    def __init__(self):
            self._DicSensor={}
    
    def regist(self, Sname, Pdata):
        if len(self._DicSensor)==7:
            print "this class has max data"
        else:
            data = SensorData()
            data.ImportCSV(Sname,Pdata)   
            self._DicSensor[Sname]=data
            
    def ExecFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcFFT(samp)

    def DispFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetFFT('fft_'+axis,samp)    
            
    def ExecPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcPower(samp)

    def DispPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetPower('power_'+axis,samp)
            
    def ShowData(self,Sname):
        print self._DicSensor[Sname].ShowAllDf()
        
    # Kallback Librar Divergence
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def ExecKLD(self,Sarray1,Saxis1,Sarray2,Saxis2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowKLD(self,Sarray1,Saxis1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
# Janson Shanon Divergence
    def ExecJSD(self,Sarray1,Saxis1,Sarray2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        jsd = self._JSD(dist1,dist2)    
        np.savez(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        

In [5]:
class SensorData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._RawData={}    
        # fft data
        self._FFTData={}
        # power spectol data
        self._PowerData={}    
        # flag exsist data
        self._Flag_exist_data=False
        
        self._columns=['AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
        
    def ImportCSV(self,Sclass,csv_file):
        self.ClassName=Sclass
        self.Flag_exist_data=True
        # design dataframe
        data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data.Time=pd.to_datetime(data.Time)
        data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        Time=data.index.to_pydatetime().astype('datetime64[ns]')
        
        # regist each raw data 
        self._RawData['AccX'] = AccX
        self._RawData['AccY'] = AccY
        self._RawData['AccZ'] = AccZ
        self._RawData['GyrX'] = GyrX
        self._RawData['GyrY'] = GyrY
        self._RawData['GyrZ'] = GyrZ
        self._RawData['Time'] = Time
    
    def ShowFlagExistData(self):
        return self.Flag_exist_data
    
    def GetColumns(self):
        return self._columns

    def ShowAllDf(self):
        print 'AccX : ';print self._RawData['AccX']
        print 'AccY : ';print self._RawData['AccY']
        print 'AccZ : ';print self._RawData['AccZ']
        print 'GyrX : ';print self._RawData['GyrX']
        print 'GyrY : ';print self._RawData['GyrY']
        print 'GyrZ : ';print self._RawData['GyrZ']
        
    def _Time2Num(self,time):
        return np.where(self._RawData['Time']==np.datetime64(time) )[0][0]
    
    def ShowQuery(self,Sname,rng=[]):
        data = self._RawData[Sname]
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._RawData['Time'][0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._RawData[Sname][start:goal]
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._RawData[Sname][start:goal]
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break  
                
# Fast Frier transaction            
    def GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+self.ClassName+'_'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        start = time.time()
        fft_data = np.array([])
        
        for n,f in zip( self._columns,self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            while True:
                try:
                    d=sw.next()
                    fft_data = np.append(fft_data, fftpack.fft(d)[1:(samp/2)+1] )   # fftの直流成分を除くsample/2の
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    fft_data = fft_data.reshape(len(fft_data)/(samp/2),(samp/2) )
                    self._FFTData[f] = fft_data
                    np.savez(p_path+'fft/'+self.ClassName+'_'+str(f)+'_'+str(samp),self._FFTData[f])
                    break   
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
           

# Spectol Power 
    def GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+self.ClassName+'_'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        start = time.time()
        for fft_name,power_name in zip( self._fft_col, self._power_col):
            print 'start'+fft_name+'->'+power_name
            fft_data = self.GetFFT(fft_name,samp)
            power_data=self._power(fft_data)
            np.savez(p_path+'power/'+self.ClassName+'_'+power_name+'_'+str(samp),power_data)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

作成したクラスのデータをテストする

class.regist(検索ワード,ファイル名)　クラスにファイルを追加する

In [7]:
AllTest = AllSensorData()
AllTest.regist('left_hand','Conv-left-hand-ags.csv')
AllTest.regist('right_leg','Conv-right-leg-ags.csv')
AllTest.regist('left_leg','Conv-left-leg-ags.csv')
AllTest.regist('right_hand','Conv-right-hand-ags.csv')

__class__
__class__
__class__
__class__


class.ShowData(検索ワード)　クラスに追加したファイルを表示する

In [8]:
AllTest.ShowData('left_hand')

AccX : 
[ 0.8226  0.8058  0.8292 ..., -0.1397 -0.1346 -0.1292]
AccY : 
[ 0.7922  0.8437  0.9091 ...,  0.7067  0.7526  0.8188]
AccZ : 
[-0.0543 -0.0582 -0.0231 ..., -0.5123 -0.4901 -0.5248]
GyrX : 
[ 353.11  361.41  353.95 ...,   -4.59   -2.53   -2.67]
GyrY : 
[ 71.96  59.74  48.03 ...,  56.    57.86  60.15]
GyrZ : 
[ -9.53 -22.7  -34.02 ..., -36.9  -32.73 -29.73]
None


In [9]:
AllTest.ShowData('left_leg')

AccX : 
[ 0.9501  0.9526  0.9584 ...,  0.9294  0.967   0.8093]
AccY : 
[ 0.1694  0.1709  0.1829 ..., -0.454  -0.5392 -0.1962]
AccZ : 
[-0.0438 -0.0521 -0.0445 ...,  0.5364  0.5989 -0.1353]
GyrX : 
[  6.08   7.53   9.67 ...,  62.94  71.18  83.5 ]
GyrY : 
[-15.73 -16.93 -18.4  ...,  37.97  54.34  46.68]
GyrZ : 
[-5.98 -6.38 -6.61 ..., -8.83  0.36 -5.52]
None


In [10]:
AllTest.ShowData('right_leg')

AccX : 
[ 0.8565  0.8391  0.8926 ...,  0.9554  0.9534  0.9619]
AccY : 
[ 0.0087 -0.0596 -0.0813 ..., -0.28   -0.2688 -0.2673]
AccZ : 
[ 0.209   0.157   0.1709 ...,  0.0203  0.0527  0.0481]
GyrX : 
[-21.33 -23.51 -14.14 ...,   7.81   7.21   6.54]
GyrY : 
[ -4.79   7.47  17.92 ...,   6.63   5.67   5.02]
GyrZ : 
[ 13.29   5.15  -5.34 ...,  -7.26  -7.03  -6.23]
None


In [11]:
AllTest.ShowData('right_hand')

AccX : 
[ 0.8258  0.8805  0.9398 ...,  0.0316  0.0463  0.0057]
AccY : 
[-0.3066 -0.2997 -0.2975 ..., -0.0693 -0.1137 -0.0734]
AccZ : 
[ 0.1603  0.1664  0.1911 ...,  1.1037  1.0844  1.0234]
GyrX : 
[-158.22 -152.1  -146.42 ...,  -13.93  -15.14  -11.08]
GyrY : 
[-18.43 -23.89 -28.76 ..., -13.11 -12.04  -9.8 ]
GyrZ : 
[-87.73 -89.44 -92.54 ...,  42.57  42.61  41.56]
None


class.ExecFFT(検索ワード,ウィンドウ幅orデータのサンプル数)
FFTを実行する

In [12]:
AllTest.ExecFFT('left_hand',16)
AllTest.ExecFFT('left_leg',16)

left_hand
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
elapsed_time:6.03894305229[sec]
left_leg
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
elapsed_time:5.98586606979[sec]


In [13]:
AllTest.ExecFFT('right_leg',16)
AllTest.ExecFFT('right_hand',16)

right_leg
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
elapsed_time:5.82902503014[sec]
right_hand
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
elapsed_time:5.93414902687[sec]


class.DispFFT(検索ワード,ウィンドウ幅orデータのサンプル数)
FFTした結果を表示する

In [15]:
AllTest.DispFFT('left_hand',16)

left_hand
[[ 0.37920144-0.05929202j -0.05606245-0.02899706j  0.04128879-0.0864666j
  ...,  0.01606245+0.02560294j  0.05219851+0.06089044j  0.11930000+0.j        ]
 [ 0.09903043-0.92279313j  0.27451966-0.49979698j  0.15211482-0.28934496j
  ...,  0.17708034-0.09459698j  0.16682011-0.03510812j  0.19270000+0.j        ]
 [ 0.38919818+0.18367701j  0.01575534+0.14228835j -0.00706838+0.11749315j
  ..., -0.05495534+0.03448835j -0.04392128-0.00738628j -0.05050000+0.j        ]
 ..., 
 [-0.41038202-0.19909711j -0.37146435+0.13109855j  0.06954730+0.09766265j
  ..., -0.06613565+0.00989855j -0.03534312-0.02376532j -0.08550000+0.j        ]
 [ 0.25773225+0.66071007j -0.25461936+0.26735258j -0.01388304+0.20446779j
  ..., -0.08958064+0.05155258j -0.09345433+0.02243494j -0.12290000+0.j        ]
 [ 0.30710427+0.04456604j  0.02138894-0.0667804j   0.02723755+0.03917587j
  ..., -0.02018894+0.0826196j  -0.06477277-0.07910414j  0.08690000+0.j        ]]
[[ 0.37920144-0.05929202j -0.05606245-0.02899706j  0.041288

In [16]:
AllTest.DispFFT('left_leg',16)

left_leg
[[-0.05430379 +1.41873617e-02j -0.00152197 -9.13047294e-02j
   0.02561294 -1.57209102e-04j ..., -0.05087803 +2.46952706e-02j
  -0.01050739 -1.79984248e-02j  0.03450000 +0.00000000e+00j]
 [ 0.00398147 +3.13533492e-01j -0.04101243 +2.64053798e-01j
  -0.10123079 +8.25556837e-02j ..., -0.10818757 +6.30537985e-02j
  -0.08432004 +4.01623882e-02j -0.04840000 +0.00000000e+00j]
 [ 0.12887828 +5.31425453e-01j  0.13709350 +3.81140003e-01j
  -0.12627790 +1.96792441e-01j ..., -0.20189350 +3.87400033e-02j
  -0.18833976 -4.71151671e-05j -0.15690000 +0.00000000e+00j]
 ..., 
 [-0.55308459 +7.98166815e-01j -0.08130969 +2.25691143e-01j
  -0.15218231 +1.57155629e-01j ..., -0.09729031 -1.85088566e-02j
  -0.06821343 -1.87749232e-02j -0.09950000 +0.00000000e+00j]
 [-0.23377219 -4.14532004e-01j  0.03361918 +3.90612265e-02j
  -0.02743281 -2.32508283e-01j ...,  0.11338082 +2.48612265e-02j
  -0.02037739 -1.70189376e-02j  0.01080000 +0.00000000e+00j]
 [-0.19055571 +3.87255994e-02j  0.02774335 +3.51379726

In [17]:
AllTest.DispFFT('right_leg',16)

right_leg
[[ 0.18918139+0.53636481j -0.10848650+0.23996887j -0.18146697+0.18075618j
  ..., -0.08331350+0.08416887j -0.07049860+0.01898412j -0.08560000+0.j        ]
 [-0.29460938-0.40254504j  0.02090725-0.16157155j  0.05883569-0.09217763j
  ...,  0.05329275-0.02977155j  0.06774223-0.00630691j  0.05470000+0.j        ]
 [ 0.87916669+1.30013644j -1.05767381+1.11453425j -1.03616793-0.52221813j
  ...,  0.52907381-0.04906575j -0.05488085+0.2882013j   0.15530000+0.j        ]
 ..., 
 [-0.04969063-0.02721006j  0.02955914+0.12793114j  0.07629629-0.00991074j
  ..., -0.03195914+0.02353114j -0.02192056+0.00235949j -0.02570000+0.j        ]
 [ 0.22240105-0.18728499j  0.04075772-0.10320078j  0.03319202-0.05019173j
  ...,  0.03524228-0.00640078j  0.05850273-0.02156943j  0.05690000+0.j        ]
 [-0.22815089+0.05091498j  0.07373940+0.01295097j -0.01489403+0.03022817j
  ..., -0.00573940+0.00515097j -0.00989675+0.02194959j  0.00670000+0.j        ]]
[[ 0.18918139+0.53636481j -0.10848650+0.23996887j -0.18146

In [19]:
AllTest.DispFFT('right_hand',16)

right_hand
[[-0.45982319+0.68872581j -0.36633099+0.3341367j  -0.19361401+0.25788001j
  ..., -0.18106901+0.0751367j  -0.17933127+0.04957271j -0.17620000+0.j        ]
 [ 0.19889739-0.54039914j  0.05762477-0.18230571j  0.05213918-0.11751329j
  ...,  0.07657523-0.04170571j  0.08424588-0.0063013j   0.10220000+0.j        ]
 [ 0.11076755+0.06095781j  0.01538808+0.05582361j -0.00544721+0.02228161j
  ..., -0.03198808-0.00137639j -0.01388773+0.01919313j -0.00100000+0.j        ]
 ..., 
 [ 0.81745523-0.43096431j  0.61408931-0.64506799j  0.82149424-0.45956686j
  ...,  0.08291069+0.11813201j  0.57034805-0.40957525j  0.02830000+0.j        ]
 [-0.42094338+0.55120469j -0.15981041+0.23798437j -0.08467249-0.08384312j
  ..., -0.14778959+0.17878437j -0.11832490-0.01352289j -0.11110000+0.j        ]
 [-0.17221011+0.10663402j -0.00823627+0.26120619j -0.04777862-0.05460637j
  ..., -0.17016373+0.14000619j -0.04831151+0.02028154j  0.06790000+0.j        ]]
[[-0.45982319 +6.88725806e-01j -0.36633099 +3.34136702e-0

class.ExecPower(検索ワード,サンプル数)
パワースペクトルの計算

In [20]:
AllTest.ExecPower('left_hand',16)
AllTest.ExecPower('right_leg',16)
AllTest.ExecPower('left_leg',16)
AllTest.ExecPower('right_hand',16)

left_hand
startfft_AccX->power_AccX
startfft_AccY->power_AccY
startfft_AccZ->power_AccZ
startfft_GyrX->power_GyrX
startfft_GyrY->power_GyrY
startfft_GyrZ->power_GyrZ
elapsed_time:3.64416480064[sec]
right_leg
startfft_AccX->power_AccX
startfft_AccY->power_AccY
startfft_AccZ->power_AccZ
startfft_GyrX->power_GyrX
startfft_GyrY->power_GyrY
startfft_GyrZ->power_GyrZ
elapsed_time:3.67760515213[sec]
left_leg
startfft_AccX->power_AccX
startfft_AccY->power_AccY
startfft_AccZ->power_AccZ
startfft_GyrX->power_GyrX
startfft_GyrY->power_GyrY
startfft_GyrZ->power_GyrZ
elapsed_time:3.64089894295[sec]
right_hand
startfft_AccX->power_AccX
startfft_AccY->power_AccY
startfft_AccZ->power_AccZ
startfft_GyrX->power_GyrX
startfft_GyrY->power_GyrY
startfft_GyrZ->power_GyrZ
elapsed_time:3.61064887047[sec]


class.DispPower(検索ワード,サンプル数)
パワースペクトルの表示

In [21]:
AllTest.DispPower('left_hand',16)

left_hand
[ 0.46232776  0.07603003  0.11542119 ...,  0.10343152  0.12433572
  0.10568071]
[ 0.46232776  0.07603003  0.11542119 ...,  0.05804979  0.06519015  0.064983  ]
[ 0.46232776  0.07603003  0.11542119 ...,  0.01278377  0.02964241
  0.06039579]
[ 0.46232776  0.07603003  0.11542119 ...,  0.02010032  0.005624    0.00092406]
[ 0.46232776  0.07603003  0.11542119 ...,  0.05010525  0.04767994
  0.05598797]
[ 0.46232776  0.07603003  0.11542119 ...,  0.01496412  0.02369655
  0.00806739]


In [22]:
AllTest.DispPower('left_leg',16)

left_leg
[ 0.16275935  0.26480835  0.0742755  ...,  0.04539268  0.06530898
  0.01775142]
[ 0.16275935  0.26480835  0.0742755  ...,  0.08998267  0.06095294
  0.04824081]
[ 0.16275935  0.26480835  0.0742755  ...,  0.06917866  0.07648995
  0.07865896]
[ 0.16275935  0.26480835  0.0742755  ...,  0.05490231  0.0544099
  0.05472876]
[ 0.16275935  0.26480835  0.0742755  ...,  0.00552456  0.00363254
  0.0019921 ]
[ 0.16275935  0.26480835  0.0742755  ...,  0.05528338  0.04964899
  0.05048234]


In [23]:
AllTest.DispPower('right_leg',16)

right_leg
[ 0.33154776  0.15351877  0.1493092  ...,  0.01825159  0.05698402
  0.01585678]
[ 0.33154776  0.15351877  0.1493092  ...,  0.05726374  0.08618664
  0.02859596]
[ 0.33154776  0.15351877  0.1493092  ...,  0.06389513  0.07302561
  0.04852646]
[ 0.33154776  0.15351877  0.1493092  ...,  0.06144643  0.05784436
  0.05711233]
[ 0.33154776  0.15351877  0.1493092  ...,  0.04037917  0.04440252
  0.03179366]
[ 0.33154776  0.15351877  0.1493092  ...,  0.04347318  0.05263045
  0.04740187]


In [24]:
AllTest.DispPower('right_hand',16)

right_hand
[ 0.31254463  0.18713317  0.12170602 ...,  0.16039295  0.0381378
  0.04942278]
[ 0.31254463  0.18713317  0.12170602 ...,  0.05524016  0.05748465
  0.00784988]
[ 0.31254463  0.18713317  0.12170602 ...,  0.09888579  0.10848537
  0.04666228]
[ 0.31254463  0.18713317  0.12170602 ...,  0.04660753  0.07995627
  0.05472128]
[ 0.31254463  0.18713317  0.12170602 ...,  0.05376991  0.0517683
  0.05327623]
[ 0.31254463  0.18713317  0.12170602 ...,  0.09345732  0.07749855
  0.05449304]


In [ ]:
AllTest.ExecKLD('left_hand','AccX','righ_hand','AccX',samp=16)

In [ ]:
import multiprocessing as mp

In [ ]:
class AllSensorData:        
    def __init__(self):
            self._DicSensor={}
    
    def regist(self, Sname, Pdata):
        if len(self._DicSensor)==7:
            print "this class has max data"
        else:
            data = SensorData()
            data.ImportCSV(Sname,Pdata)   
            self._DicSensor[Sname]=data
            
    def ExecFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcFFT(samp)

    def DispFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetFFT('fft_'+axis,samp)    
            
    def ExecPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcPower(samp)

    def DispPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetPower('power_'+axis,samp)
            
    def ShowData(self,Sname):
        print self._DicSensor[Sname].ShowAllDf()
        
    # Kallback Librar Divergence
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def ExecKLD(self,Sarray1,Saxis1,Sarray2,Saxis2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        proc = 8
        
        def subcalc(p):
            ini = len(dist)*p/proc
            fin = len(dist)*p/proc
            
            jsd = self._KLD(dist1[ini,fin],dist2[ini,fin]) 
            return jsd
        
        pool = mp.Pool(proc)
        callback = pool.map(subcalc, range(8))
        
        total = callback.reshape(1,)
        
        kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowKLD(self,Sarray1,Saxis1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
# Janson Shanon Divergence
    def ExecJSD(self,Sarray1,Saxis1,Sarray2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        proc = 8
        
        def subcalc(p):
            ini = len(dist)*p/proc
            fin = len(dist)*p/proc
            
            jsd = self._JSD(dist1[ini,fin],dist2[ini,fin]) 
            return jsd
        
        pool = mp.Pool(proc)
        callback = pool.map(subcalc, range(8))
        
        total = callback.reshape(1,)
        
        np.savez(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,total)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        

In [ ]:
AllTest.ExecKLD('left_hand','AccX','righ_hand','AccX',samp=16)